# Задание 1 (10 баллов)

Вам нужно воспроизвести пайплайн дообучения из семинара, используя русскоязычные модели. То есть, вам нужно дообучить генеративную модель генерировать более положительные тексты с помощью модели определения тональности.

1) Вам потребуется модель определения тональности для русского языка. Вы можете найти ее на huggingface или обучить самостоятельно. В прошлых семинарах мы пользовались датасетом с токсичными текстами - можно использовать его для обучения (или любой другой датасет на русском языке). Удобнее всего будет обучать модель через huggingface. Можете взять за основу вот этот туториал - https://huggingface.co/docs/transformers/training#train-a-tensorflow-model-with-keras После обучения можете сохранить модель через model.save_pretrained(path) и потом использовать ее как любую другую модель из huggingface hub.
Используйте не очень большую модель, чтобы все поместилось в колабе (например, distilbert-multilingual)

In [1]:
!pip3 install scikit-learn pandas matplotlib transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00


In [2]:
import os, re
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import Counter
from string import punctuation
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
%matplotlib inline

import tensorflow as tf
from transformers import TFAutoModel
from transformers import AutoTokenizer

In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.6 MB/s eta 0:00:00


In [5]:
! pip install trl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 10.0 MB/s eta 0:00:00


In [6]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

Берём генеративную модель ai-forever/rugpt3small_based_on_gpt2

In [7]:
# тут задаются гиперпараметры
config = PPOConfig(
    model_name="ai-forever/rugpt3small_based_on_gpt2",
    learning_rate=1.41e-5,
    log_with=None,
    mini_batch_size=16
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

Для генерирования текстов используем MonoHime/ru_sentiment_dataset с Huggingface. От каждого текста отрезается случайно какой-то кусочек, и модель должна его продолжить.

In [8]:
from google.colab import drive
drive.mount('/content/drive')

dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Colab Notebooks/labeled.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4310d66bb18c9da6/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['comment', 'toxic'],
        num_rows: 14412
    })
})

In [10]:
def build_dataset(config,
                  dataset_name=dataset['train'],
                  input_min_text_length=2,
                  input_max_text_length=8):
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    ds = dataset
    ds = ds.rename_columns({"comment": "review"})

    # еще есть фильтрация по длине
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)
    ds = ds.filter(lambda x: len(x["review"]) < 2000, batched=False)

    # длина кусочка определяется случайно
    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds
dataset = build_dataset(config)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Filter:   0%|          | 0/14412 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3511 [00:00<?, ? examples/s]

Map:   0%|          | 0/3454 [00:00<?, ? examples/s]

In [11]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [12]:
# active_model
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
# reference_model (обратите внимание что это одна и так же модель изначально)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
ppo_trainer = PPOTrainer(config=config, model=model, ref_model=ref_model, tokenizer=tokenizer,
                         dataset=dataset["train"], data_collator=collator)

In [14]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

Для оценки тональности используется пайплайн из huggingface и предобученная модель

In [15]:
pip install xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 10.7 MB/s eta 0:00:00


In [16]:
sentiment_pipe = pipeline("sentiment-analysis", model="cointegrated/rubert-tiny-sentiment-balanced", device=device)

In [17]:
text = "Это ужасный фильм!!"
sentiment_pipe(text, **sent_kwargs)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'negative', 'score': 0.46191224455833435},
  {'label': 'neutral', 'score': -0.8093413710594177},
  {'label': 'positive', 'score': 0.22957782447338104}]]

In [18]:
text = "Шикарное кино!!"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'negative', 'score': -3.250540018081665},
  {'label': 'neutral', 'score': -0.664109468460083},
  {'label': 'positive', 'score': 3.555371046066284}]]

In [19]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0,
              "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

In [20]:
import torch
torch.cuda.empty_cache()

In [21]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader),
                         total=dataset["train"].num_rows//ppo_trainer.dataloader.batch_sampler.batch_size):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[2]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

  0%|          | 0/13 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [22]:
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_colwidth', 5000)

In [24]:
#### get a batch from the dataset
bs = 16
game_data = dict()
dataset['train'].set_format("pandas")
df_batch = dataset['train'][:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:].to(device)
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:].to(device)
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [25]:
pd.DataFrame.from_dict(game_data)

,query,response (before),response (after),rewards (before),rewards (after)
0,Комму,"низмов"" в Ижевске на",няшка) \n 21 октября 2014,3.011113,3.035393
1,"Уважаемый, я непременно",им стану. Я никогда и ни к,"вас программу ""просматриваю"") люблю",0.458643,-0.608372
2,Ну да бог с ними.,"Боб прикончил — а стало быть, душа живая",Мне даже очень хорошо. Хорошо. \n,-0.480960,-0.743648
3,"Понимаете, тут","у меня случилась личная встреча. Я присев на стул, уставился на Гре",так вкусно и много фруктов!! спасибочки большое!)\nЯ вся довольна.,1.332774,-1.399627
4,От неё и не,"замечаю.\nОн, видимо, раз",только мне) ) будет хорошо) ) обожа,-0.263283,-0.317939
5,Они не учли,одного факта: в военной службе украинцы,"одну важную особенность). - То,",3.114699,0.848219
6,Блядь,кем мы были сегодня - сукиным сыном или бандитом?,", снег же! Может, че там сегодня-то",3.268763,0.994380
7,Допу,"стим, Вы вспомнили о существовании системы GPS муниципального образования """,стим). В такой фирме каждая единица вознаграждалась отдельно,3.749329,0.074258
8,В заголовке какой-,"то знак вопроса и коммент ""рыба"" с плохим",либо серии интересно было почитать)). Номинации супер,1.735664,0.327291
9,Могли,"говорить о Вас? Я понимаю: различные суждения, суждения, тоже","такое моменты\nРастратали хорошо, теперь ходим хорошо\n\n",0.239845,-0.418318
